# Sistema di produzione di neve artificiale
---

## Formulazione del problema

- orizzonte temporale: $t=0,1,...T$, $T = 72$ ore considerando un time step orario
- stato $x_t: (temperatura\_ora\_t,umidità\_ora\_t,livello\_neve\_ora\_t) = (T_t,RH_t,d_t) = (i,j,k)$
- spazio degli stati $x_t \in X_t=\{(i,j,k): i=1,...,50, j=1,...,50, k=1,...,n\}$
    - $T_t \in\{T_1,...,T_{50}\}_t$ con valori diversi da ora in ora
        - $T_t$ variabile aleatoria discreta con pmf $P(T_t=T_i)_t$
    - $RH_t \in\{RH_1,...,RH_{50}\}_t$ con valori diversi da ora in ora
        - $RH_t$ variabile aleatoria discreta con pmf $P(RH_t=RH_i)_t$
    - $d_t \in D=\{0,...,max_d\}, |D|=n$
- ingresso $u_t=(af_t,wf_t)$
    - $af_t$: flusso di aria iniettato nella macchina sparaneve nell'ora t $[m^3/h]$
    - $wf_t$: flusso di acqua iniettato nella macchina sparaneve nell'ora t $[m^3/h]$
- spazio degli ingressi $u_t \in U= 
    \begin{cases}
    \{(0,0)\} & \text{se } \text{la } \text{neve } \text{non } \text{può } \text{essere } \text{prodotta } \text{dati } (T_t,RH_t) \\
    \{(min_{af},min_{wf}),...,(max_{af},max_{wf})\} & \text{altrimenti } 
    \end{cases}$
    - $min_{wf} = 0.6 [m^3/h], max_{wf} = [3.6,5.4] [m^3/h]$
        - step di discretizzazione: $0.2$
    - $min_{af} = 6 [m^3/h], max_{af} = 27 [m^3/h]$
        - step di dicretizzazione: $1$
    - la neve non può essere prodotta se la temperatura del bulbo umido è superiore alla soglia operativa $T_{wb,soglia} = -2.5 °C$
- matrice di transizione per le componenti di temperatura e umidità dello stato
    - $P((T_{t+1}=i,RH_{t+1}=j)|(T_{t}=i',RH_{t}=j'))=P(T_{t+1}=i|T_t=i')_{t+1}*P(RH_{t+1}=j|RH_t=j')_{t+1}$
- funzione di transizione per la componente "livello_neve" dello stato
    - $d_{t+1}=d_t+Neve\_prodotta(x_t,u_t)-Neve\_fusa(x_t)$
    - valore da approssimare al valore discreto più vicino assunto da $d_t$
- funzione per determinare la neve prodotta in un time step in $m^3$
    - $Neve\_prodotta(T_t,RH_t,af_t,wf_t)=k(T_t,RH_t)*min(af_t,wf_t)$
    - $k=\alpha * max(0,T_{wb,soglia}-T_{wb})$ coefficiente di efficienza
    - $\alpha:$ coefficiente calibrato $\in [0.4,1]$
    - $T_{wb,soglia} = -2.5 °C$: soglia (wet-bulb) operativa del generatore
    - $T_{wb} ≈ T_t ⋅arctan(0.151977⋅(RH_t+8.313659)^{0.5})+arctan(T_t+RH_t)-arctan(RH_t−1.676331)
+0.00391838⋅RH^{1.5}⋅arctan(0.023101⋅RH)−4.686035$
        - dove $T_t$ in $°C$ e $RH_t$ in percentuale
- funzione per determinare la neve fusa in un time step in $m^3$
    - $Neve\_fusa(x_t)= (DDF/(1000*24)) * max(T_{avg}-T_0,0) * A$
        - $DDF = degree-day-factor (mm/°C/giorno)$ (tipicamente 1–10 mm)
        - $T_{avg} = temperatura-media °C$
        - $T_0 = soglia (≈°C)$
        - $A = area (m^2)$
- stage cost $g(x_t,u_t):$ deterministico, tempo-invariante
    - $g(x_t,u_t)=prezzo\_energia*(E_{ventola}+E_{compressore}(u_t)+E_{pompa}(u_t))$
    - $prezzo\_energia$ in kWh
    - $E_{ventola}=18.5 kWh$ costante
    - $E_{compressore}(u_t)= e * af_t * 60$
        - $e=6.2 [kW/(m^3/min)]$
        - $af_t [m^3/h]$
    - $E_{pompa}(u_t)=k_{pump}*wf_t$
        - $k_pump=3.5 [kWh/m^3]$
        - $af_t [m^3/h]$
- Terminal cost $g_T(x_T)$:
    - $g_T((i,j,k)) = \begin{cases} 0 & \text{if } k = n \\ \infty & \text{if } k \neq n \end{cases}$
    - costo infinito se non viene prodotta $max_d [m^3]$ di neve al termine dell'orizzonte temporale

## Formulazione del problema
---

### Importazione moduli

In [1]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import json
import math 
from functions import *

### Caricamento dati meteorologici da file

In [2]:
# CARICAMENTO DATI temperatura e umidità DA FILE JSON
with open("pmf_temperature.json", "r", encoding="utf-8") as f1:
    temperature = json.load(f1)

with open("pmf_humidity.json", "r", encoding="utf-8") as f2:
    humidity = json.load(f2)

# conversione dei dizionari in liste
#temperatura
temp_val=[] #valori discreti per fasce orarie
temp_pmf=[] #pmf per fasce orarie
for fasce,dict_val in temperature.items():
    temp_val.append(np.array(dict_val['values']))
    temp_pmf.append(np.array(dict_val['pmf']))
    
#umidità
humid_val=[] #valori discreti per fasce orarie
humid_pmf=[] #pmf per fasce orarie
for fasce,dict_val in humidity.items():
    humid_val.append(np.array(dict_val['values']))
    humid_pmf.append(np.array(dict_val['pmf']))

#caricamento delle matrici delle probabilità condizionate da file
P_T_list = load_transition_matrices("transition_T.json") #lista di matrici P_T, una per ogni fascia oraria
P_RH_list = load_transition_matrices("transition_RH.json") #lista di matrici P_RH, una per ogni fascia oraria

print("Caricate", len(P_T_list), "matrici di transizione T")
print("Caricate", len(P_RH_list), "matrici di transizione RH")
#print("Dimensione prima matrice T:", P_T_list[0].shape)
#print(P_T_list[0])

Caricate 24 matrici di transizione T
Caricate 24 matrici di transizione RH


### Parametri del problema

In [3]:
n_fasce=24 #fasce orarie
n_T = temp_val[0].shape[0] #numero di intervalli di discretizzazione di temperatura
n_RH = humid_val[0].shape[0] #numero di intervalli di discretizzazione di umidità
n_T_RH = n_T * n_RH #numero di possibili combinazioni di temperatura e umidità per fascia oraria
max_d=100 #profondità massima della neve
step_d=0.5 #step di discretizzazione della profondità della neve
#flusso di aria minimo e massimo in ingresso allo sparaneve
min_af=6
max_af=27
af_step=1 #step di discretizzazione del flusso di aria
#flusso di acqua minimo e massimo in ingresso allo sparaneve
min_wf=0.6
max_wf=4.5 #[3.6,5.4]
wf_step=0.2 #step di discretizzazione del flusso di acqua

### Formulazione del problema 

In [4]:
# Definizione dello spazio dei valori discreti della profondità della neve
depth_space=np.arange(0,max_d+step_d,step_d)
n_d=depth_space.shape[0] ##numero di intervalli di discretizzazione della profondità della neve

# Spazio degli stati
state_space=[]
n_states=n_T*n_RH*n_d #numero di stati per fascia oraria
# - definizione di uno spazio degli stati per ogni fascia oraria
for f in range(n_fasce):
    X=np.empty((n_T,n_RH,n_d,3))
    for i,T in enumerate(temp_val[f]):
        for j,RH in enumerate(humid_val[f]):
            for k,d in enumerate(depth_space):            
                X[i,j,k]=[T,RH,d]
    state_space.append(X)

# Spazio degli ingressi
U=input_space(-1,90,min_af, max_af, min_wf, max_wf, af_step, wf_step)

# Calcolo matrici P_t di transizione delle componenti temperatura e umidità dello stato
# --> P_list conterrà una matrice di transizione per fascia oraria
P_list = []

for f in range(n_fasce):
    P_T = P_T_list[f] #matrice delle probabilità condizionate della temperatura relativa alla fascia f
    P_RH = P_RH_list[f] #matrice delle probabilità condizionate dell'umidità relativa alla fascia f
    P_list.append(build_joint_transition(P_T, P_RH))

print(P_list[0].shape)

# Dinamica del sistema
# - funzione che restituisce uno stato iniziale secondo le distribuzioni di temperatura e umidità nella fascia oraria iniziale
# - e secondo la profondità iniziale della neve passata come parametro
def get_initial_state(f_t0,d0):
    # --> f_t0: fascia oraria dell'istante di tempo iniziale t0 del time horizon
    # da aggiungere il controllo della fascia oraria
    pmf_T_t0=temp_pmf[f_t0] # pmf temperatura nella fascia f_t0
    pmf_RH_t0=humid_pmf[f_t0] # pmf umidità nella fascia f_t0
    T0=np.random.choice(temp_val[f_t0], p=pmf_T_t0) # estrazione di un campione di temperatura dalla relativa pmf
    RH0=np.random.choice(humid_val[f_t0], p=pmf_RH_t0) # estrazione di un campione di umidità dalla relativa pmf
    x_0=[T0,RH0,d0]
    return x_0
print(get_initial_state(0,depth_space[-1]))

# - funzione di transizione per la componente 'profondità_neve' dello stato (deterministica)
def depth_transition(x_t,u_t):
    T_t=x_t[0]
    RH_t=x_t[1]
    d_t=x_t[2]
    af_t=u_t[0]
    wf_t=u_t[1]
    # profondità della neve al prossimo time step (fascia oraria)
    d_next=d_t+snow_produced(T_t,RH_t,af_t,wf_t)-snow_melted(T_t,RH_t)
    if d_next<0:
        d_next_discretized=np.float64(0)
    else: # arrotondamento al campione discreto più vicino
        d_next_discretized=depth_space[np.digitize(d_next,depth_space)-1]
    return d_next_discretized

x_t=[-13.1757,30.776318339275747,5]
u_t=[10,3]
print(depth_transition(x_t,u_t))

# - evoluzione di uno step per temperatura e umidità, secondo la matrice di transizione realtiva alla fascia f
def weather_transition(x_t,f): #da aggiungere la rotazione delle fasce --> 23 - 0
    T_t=x_t[0]
    RH_t=x_t[1]
    # matrice di transizione relativa alla fascia f
    P_f = P_list[f] 
    # conversione stato in indice
    state_idx = state_to_index(T_t,RH_t,temp_val[f],humid_val[f])
    # estrazione di un campione secondo la pmf condizionata allo stato corrente, presente nella matrice P_f
    next_state_idx = np.random.choice(np.arange(0,n_T_RH,1), p=P_f[state_idx]) 
    # conversione indice ottenuto in stato
    if f<23:
        next_T, next_RH = index_to_state(next_state_idx,temp_val[f+1],humid_val[f+1])
    else: #siamo nell'ultima fascia oraria, i prossimi valori da considerare per temperatura e umidità vengono presi dalla fascia 0
        next_T, next_RH = index_to_state(next_state_idx,temp_val[0],humid_val[0])
    return next_T, next_RH
print(weather_transition(x_t,0))

# - simulazione del sistema per un singolo time horizon t=0,...,T
def simulate(T,d0):
    # --> T: lunghezza time horizon
    # --> d0: profondità della neve all'inizio del time horizon
    X_sim=[] #lista degli stati visitati dal sistema durante la simulazione
    f_t0=0 #fascia oraria a cui appartiene t0
    x_0=get_initial_state(f_t0,d0) 
    # inizializzazione
    u_t=[0,0]
    x_t=x_0
    f_t=f_t0
    # iterazione
    for t in range(T):
        next_T, next_RH = weather_transition(x_t,f_t)
        next_d = depth_transition(x_t,u_t)
        x_tp1 = [next_T,next_RH,next_d]
        X_sim.append(x_tp1)
        print("time "+str(t)+", fascia "+str(f_t)+", stato: "+str(x_tp1))
        x_t=x_tp1
        f_t=(t+1)%24

simulate(72,depth_space[-1])
                                             

(2500, 2500)
[np.float64(0.5297000000000001), np.float64(96.28148899179524), np.float64(100.0)]
29.5
(np.float64(-12.827100000000002), np.float64(61.8720685597475))
time 0, fascia 0, stato: [np.float64(-2.7555000000000005), np.float64(46.8678259872975), np.float64(100.0)]
time 1, fascia 1, stato: [np.float64(-2.44205), np.float64(44.29708995040625), np.float64(100.0)]
time 2, fascia 2, stato: [np.float64(-2.310625), np.float64(40.54352917101501), np.float64(100.0)]
time 3, fascia 3, stato: [np.float64(-1.690199999999999), np.float64(75.96919398028375), np.float64(100.0)]
time 4, fascia 4, stato: [np.float64(-2.373999999999998), np.float64(78.782975605009), np.float64(100.0)]
time 5, fascia 5, stato: [np.float64(-2.4648249999999994), np.float64(77.507341734549), np.float64(100.0)]
time 6, fascia 6, stato: [np.float64(-3.1035000000000004), np.float64(74.73494257748025), np.float64(100.0)]
time 7, fascia 7, stato: [np.float64(-2.8256249999999996), np.float64(63.11226093644125), np.float64